In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#pip install git@github.com:badw/SMILESBOX.git
#pip install git@github.com:badw/reactit.git
#pip install mace-torch

In [54]:
from ase.optimize import QuasiNewton
from ase.vibrations import Vibrations
from smilesbox.smilesbox import SMILESbox
from arcs.generate import GetEnergyandVibrationsAseCalc,GetEnergyandVibrationsVASP
import shutil
from mace.calculators import MACECalculator 
from mace.calculators import mace_off

For generating ARCS data using GNNs or ML Models (Compatible with ASE)


we can set up a reaction graph between the following reagents:

1. H2,
2. O2,
3. H2O
4. H2O2

In [43]:
reagents = {'H2':'[HH]','O2':'O=O','H2O':'O','H2O2':'OO'}

from reactit.reactit import ReactionGenerator

rg = ReactionGenerator(compounds=list(dft_dict))

reactions = rg.iterate()
reactions

100%|██████████| 11/11 [00:00<00:00, 307.21it/s]


['2 H2 + 1 O2 = 2 H2O',
 '1 H2 + 1 H2O2 = 2 H2O',
 '1 H2 + 1 O2 = 1 H2O2',
 '1 O2 + 2 H2O = 2 H2O2']

In [40]:
#calc = CHGNetCalculator(use_device='mps')

calc = mace_off(model='large')

dft_dict = {}
for reagent in reagents:
    print(reagent)
    try:
        shutil.rmtree('./vib/')
    except FileNotFoundError:
        pass
    sm = SMILESbox()
    sm.smiles_to_atoms(reagents[reagent])

    #sm.add_box([10,10,10])

    atoms = sm.atoms
    atoms.calc = calc

    QuasiNewton(atoms).run(fmax=0.001)

    vib = Vibrations(atoms,nfree=4) # nfree = 2 or 4 but not sure which is more accurate

    vib.run()

    gevac = GetEnergyandVibrationsAseCalc(aseatomscalc=atoms,asevibrationscalc=vib)

    dft_dict[reagent] = gevac.as_dict()



Using MACE-OFF23 MODEL for MACECalculator with /Users/badw/.cache/mace/MACE-OFF23_large.model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using head Default out of ['Default']
H2
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 15:46:22      -31.719627       1.7048


/Users/badw/miniconda3/envs/py312/lib/python3.12/site-packages/mace/calculators/mace.py:143: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


BFGSLineSearch:    1[  2] 15:46:22      -31.855335       0.6992
BFGSLineSearch:    2[  4] 15:46:22      -31.867356       0.0068
BFGSLineSearch:    3[  5] 15:46:22      -31.867357       0.0005
O2
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 15:46:23    -4091.031980       0.8976
BFGSLineSearch:    1[  1] 15:46:23    -4091.136515       0.2940
BFGSLineSearch:    2[  3] 15:46:23    -4091.137748       0.0000
H2O
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 15:46:25    -2081.116613       0.6752
BFGSLineSearch:    1[  2] 15:46:25    -2081.122018       0.0583
BFGSLineSearch:    2[  3] 15:46:25    -2081.122063       0.0104
BFGSLineSearch:    3[  4] 15:46:25    -2081.122072       0.0021
BFGSLineSearch:    4[  5] 15:46:25    -2081.122072       0.0003
H2O2
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 15:46:27    -4126.507501       1.2906
BFGSLineSearch:    1[  2]

In [44]:
from arcs.generate import GraphGenerator
dft_dict['reactions'] = rg.as_dict() # add to the dft_dict
graph = GraphGenerator().from_dict(dft_dict=dft_dict,temperature=298.15,pressure=1.0)

/Users/badw/miniconda3/envs/py312/lib/python3.12/site-packages/ase/thermochemistry.py:823: UserWarning: 1 imag modes removed
  warn(f"{n_imag} imag modes removed", UserWarning)


In [45]:
from arcs.traversal import Traversal
from arcs.generate import GenerateInitialConcentrations

gic = GenerateInitialConcentrations(graph=graph).update_ic(
    {'H2':10,'O2':10}
    )

t = Traversal(graph=graph)

data = t.sample(initial_concentrations=gic,ncpus=4,nsamples=1000)

1008it [00:03, 316.68it/s]                         


In [46]:
from arcs.analysis import AnalyseSampling
import pandas as pd 

analysis = AnalyseSampling()
stats = pd.Series(analysis.reaction_statistics(data)).sort_values(ascending=False)
stats.head(10)

2 H2 + 1 O2 = 2 H2O      1371
1 H2 + 1 O2 = 1 H2O2      685
1 O2 + 2 H2O = 2 H2O2     420
1 H2 + 1 H2O2 = 2 H2O     168
dtype: int64

In [47]:
average_data = pd.DataFrame(analysis.average_sampling(data))
average_data = average_data.loc[~(average_data==0).all(axis=1)]
average_data.sort_values(by='diff',inplace=True)
print(average_data.round(2).to_string())

      initial  mean   diff   sem   std    var
H2       10.0  0.00 -10.00  0.00  0.00   0.00
O2       10.0  3.53  -6.47  0.04  1.94   3.78
H2O2      0.0  2.94   2.94  0.08  3.89  15.10
H2O       0.0  7.06   7.06  0.08  3.89  15.10


and with DFT (in this case VASP)



In [56]:
from ase.calculators.vasp import Vasp 
import os 
os.environ['VASP_PP_PATH'] = '/Users/badw/Documents/misc/VASP/'
calc = Vasp(
    command='mpirun -np 4 /Users/badw/Documents/misc/VASP/source/vasp.6.4.2/bin/vasp_gam',
            directory='./vasp_calculations/relax/',
            kpts=(1, 1, 1),
            xc='PBE',
            encut=500,
            ediff=1e-6,
            ediffg=-1e-4,
            ibrion=1,
            isym=2,
            symprec=0.01,
            isif=2,
            nsw=100,
            lreal=False,
            lwave=False,
            lcharg=False,
            algo='All',
            potim=0.1,
            ncore=4,
            prec='accurate',
            addgrid=True,
            gamma=True
)

vibcalc = Vasp(
    command='mpirun -np 4 /Users/badw/Documents/misc/VASP/source/vasp.6.4.2/bin/vasp_gam',
            directory='./vasp_calculations/vibrations/',
            kpts=(1, 1, 1),
            xc='PBE',
            encut=500,
            ediff=1e-6,
            ediffg=-1e-4,
            ibrion=6,
            isym=2,
            symprec=0.01,
            isif=2,
            nsw=10,
            lreal=False,
            lwave=False,
            lcharg=False,
            algo='All',
            potim=0.1,
            ncore=4,
            prec='accurate',
            addgrid=True,
            gamma=True
)

dft_dict = {}
for reagent in reagents:

    if reagent == 'O2':
        calc.set(**{'ispin':2,'nupdown':2}) # triplet oxygen 
        vibcalc.set(**{'ispin':2,'nupdown':2})
    else:
        calc.set(**{'ispin':1,'nupdown':0})
        vibcalc.set(**{'ispin':1,'nupdown':0})
    
    print(reagent)
    try:
        shutil.rmtree('./vib/')
        shutil.rmtree('./vasp_calculations/')
    except FileNotFoundError:
        pass

    sm = SMILESbox()
    sm.smiles_to_atoms(reagents[reagent])
    sm.add_box([5,5,5]) #should be 10,10,10 but this is a test 
    atoms = sm.atoms
    
    atoms.calc = calc

    atoms.get_potential_energy()
    try:
        vibcalc.calculate(atoms)
    except AttributeError:
        pass 

    gevac = GetEnergyandVibrationsVASP(relax_directory='./vasp_calculations/relax/',vibrations_directory='./vasp_calculations/vibrations/')

    dft_dict[reagent] = gevac.as_dict()

H2
O2


/Users/badw/miniconda3/envs/py312/lib/python3.12/site-packages/ase/calculators/vasp/vasp.py:1004: UserWarning: Magnetic moment data not written in OUTCAR (LORBIT<10), setting magnetic_moments to zero.
Set LORBIT>=10 to get information on magnetic moments
  warn('Magnetic moment data not written in OUTCAR (LORBIT<10),'


H2O
H2O2


In [62]:
from arcs.generate import GraphGenerator
dft_dict['reactions'] = rg.as_dict()
graph = GraphGenerator().from_dict(dft_dict=dft_dict,temperature=298.15,pressure=1.0)

/Users/badw/miniconda3/envs/py312/lib/python3.12/site-packages/ase/thermochemistry.py:823: UserWarning: 1 imag modes removed
  warn(f"{n_imag} imag modes removed", UserWarning)


In [63]:
from arcs.traversal import Traversal
from arcs.generate import GenerateInitialConcentrations

gic = GenerateInitialConcentrations(graph=graph).update_ic(
    {'H2':10,'O2':10}
    )

t = Traversal(graph=graph)

data = t.sample(initial_concentrations=gic,ncpus=4,nsamples=1000)

1008it [00:05, 180.71it/s]                         


In [64]:
from arcs.analysis import AnalyseSampling
import pandas as pd 

analysis = AnalyseSampling()
stats = pd.Series(analysis.reaction_statistics(data)).sort_values(ascending=False)
stats.head(10)

2 H2 + 1 O2 = 2 H2O      2342
1 H2 + 1 O2 = 1 H2O2     1380
1 O2 + 2 H2O = 2 H2O2     716
1 H2 + 1 H2O2 = 2 H2O     514
dtype: int64

In [65]:
average_data = pd.DataFrame(analysis.average_sampling(data))
average_data = average_data.loc[~(average_data==0).all(axis=1)]
average_data.sort_values(by='diff',inplace=True)
print(average_data.round(2).to_string())

      initial  mean  diff   sem   std    var
H2       10.0  0.02 -9.98  0.00  0.03   0.00
O2       10.0  2.76 -7.24  0.03  2.45   5.99
H2O2      0.0  4.51  4.51  0.07  4.92  24.16
H2O       0.0  5.47  5.47  0.07  4.93  24.35


sample with arcs

In [66]:
pyvis_kwargs = {'width':'50%','notebook':False,"font_color":'white','directed':True}
g = analysis.result_to_pyvis(data,head=20,**pyvis_kwargs)
g.save_graph(name="example_pyvis_graph.html")

In [67]:
! open example_pyvis_graph.html 